# Import Libraries #

In [ ]:
import pandas as pd
# import numpy as np
# import math
# from datetime import datetime, timedelta
import os
# import re


# import matplotlib.pyplot as plt
# import seaborn as sns

# OLGA: Define the patient ID for processing
patient_id = 1

# Load Data #

In [ ]:
def detect_file_format(filepath):
    """
    Detect the file format and separator
    """
    # Try different separators
    separators = [',', '|', '\t', ';']
    
    for sep in separators:
        try:
            # Read first few rows to check format
            sample = pd.read_csv(filepath, sep=sep, nrows=5)
            if len(sample.columns) >= 2:  # At least 2 columns
                return sep
        except:
            continue
    
    return ','  # Default to comma

def identify_columns(df):
    """
    Identify which columns contain time and glucose data
    """
    time_col = None
    glucose_col = None
    
    # Common time column names
    time_patterns = [
        'time', 'timestamp', 'datetime', 'date', 'datadttm', 
        'data_dt_tm', 'recorded_time', 'measurement_time'
    ]
    
    # Common glucose column names
    glucose_patterns = [
        'glucose', 'cgm', 'bg', 'blood_glucose', 'sensor_glucose',
        'glucose_level', 'gl', 'sensor_value', 'reading', 'value'
    ]
    
    # Check column names (case insensitive)
    columns_lower = [col.lower() for col in df.columns]
    
    # Find time column
    for pattern in time_patterns:
        for i, col in enumerate(columns_lower):
            if pattern in col:
                time_col = df.columns[i]
                break
        if time_col:
            break
    
    # Find glucose column
    for pattern in glucose_patterns:
        for i, col in enumerate(columns_lower):
            if pattern in col:
                glucose_col = df.columns[i]
                break
        if glucose_col:
            break
    
    # If not found by name, try to identify by data type and content
    if not time_col:
        for col in df.columns:
            # Check if column contains datetime-like strings
            sample_val = str(df[col].dropna().iloc[0]) if not df[col].dropna().empty else ""
            if any(char in sample_val for char in [':', '-', '/', ' ']) and len(sample_val) > 8:
                time_col = col
                break
    
    if not glucose_col:
        for col in df.columns:
            if col != time_col:  # Don't use the time column
                try:
                    # Check if column is numeric and within reasonable glucose range
                    numeric_data = pd.to_numeric(df[col], errors='coerce').dropna()
                    if len(numeric_data) > 0:
                        min_val, max_val = numeric_data.min(), numeric_data.max()
                        # Typical glucose range is 20-600 mg/dL
                        if 20 <= min_val <= 600 and 20 <= max_val <= 600:
                            glucose_col = col
                            break
                except:
                    continue
    
    return time_col, glucose_col

def detect_timestamp_format(timestamp_str, possible_formats=None):
    """
    Try to detect timestamp with multiple possible formats
    """
    if possible_formats is None:
        possible_formats = [
            "%d%b%y:%H:%M:%S",  # Original format
            "%Y-%m-%d %H:%M:%S",
            "%m/%d/%Y %H:%M:%S",
            "%d/%m/%Y %H:%M:%S",
            "%Y-%m-%d %H:%M",
            "%m/%d/%Y %H:%M",
            "%d/%m/%Y %H:%M",
            "%Y-%m-%d",
            "%m/%d/%Y",
            "%d/%m/%Y",
            "%d-%b-%Y %H:%M:%S",
            "%d-%b-%y %H:%M:%S",
            "%Y%m%d %H:%M:%S",
            "%d%m%Y %H:%M:%S"
        ]
    
    for fmt in possible_formats:
        try:
            pd.to_datetime(timestamp_str, format=fmt)
            return fmt
        except:
            continue
    
    return None 

In [ ]:
def standardize_cgm_data(filepath, output_filename="standardized_cgm_data.csv"):
    """
    Read CGM data file and standardize it to time|glucose_level format
    """
    print(f"Processing file: {filepath}")
    
    # Detect file format
    separator = detect_file_format(filepath)
    print(f"Detected separator: '{separator}'")
    
    # Read the file
    try:
        # Try reading with headers first
        df = pd.read_csv(filepath, sep=separator)
        print(f"File loaded with {len(df)} rows and {len(df.columns)} columns")
        print(f"Columns: {list(df.columns)}")
        
        # If first row might be data (no clear headers), try skiprows=1 
        # OLGA: if your file has no header, you will need to use header=None, not skiprows=1, otheerwise you will skip the first row of data
        if len(df.columns) <= 2 and any(col.startswith('Unnamed') for col in df.columns):
            df = pd.read_csv(filepath, sep=separator, header=None, 
                           names=["col1", "col2"] if len(df.columns) == 2 else [f"col{i+1}" for i in range(len(df.columns))])
            print("Retried with header=None and custom column names")
        
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
    
    # Identify time and glucose columns
    time_col, glucose_col = identify_columns(df)
    
    if not time_col or not glucose_col:
        print("Could not automatically identify time and glucose columns.")
        print("Available columns:", list(df.columns))
        print("\nFirst few rows:")
        print(df.head())
        
        # For now, make educated guesses based on position
        if len(df.columns) >= 2:
            time_col = df.columns[0]  # First column often time
            glucose_col = df.columns[1] if len(df.columns) == 2 else df.columns[-1]  # Last column often glucose
            print(f"Using best guess - Time: {time_col}, Glucose: {glucose_col}")
    
    print(f"Identified columns - Time: {time_col}, Glucose: {glucose_col}")
    
    # Extract and clean the data
    clean_df = df[[time_col, glucose_col]].copy()
    clean_df.columns = ['time', 'glucose_level']
    
    # Remove rows with missing data
    clean_df = clean_df.dropna()
    
    # Parse timestamps
    # OLGA: do format detection using small sample of data insteadof the complete dataset
    print("Parsing timestamps...")
    timestamp_sample = clean_df['time'].head(5)
    timestamp_fmt = detect_timestamp_format(timestamp_sample)
    if timestamp_fmt:
        print(f"Detected timestamp format: {timestamp_fmt}")
        clean_df['time'] = pd.to_datetime(clean_df['time'], format=timestamp_fmt, errors='coerce')
    else:
        print("Could not detect timestamp format, using general parser")
        clean_df['time'] = pd.to_datetime(clean_df['time'],  errors='coerce')    
    
    # Remove rows where timestamp parsing failed
    clean_df = clean_df[clean_df['time'].notna()]
    
    # Convert glucose to numeric
    clean_df['glucose_level'] = pd.to_numeric(clean_df['glucose_level'], errors='coerce')
    clean_df = clean_df[clean_df['glucose_level'].notna()]
    
    # Remove duplicates and sort by time
    clean_df = clean_df.drop_duplicates()
    clean_df = clean_df.sort_values('time')

    #OLGA: drop index
    clean_df.reset_index(drop=True, inplace=True)
    
    # Validate glucose values (reasonable range)
    initial_count = len(clean_df)
    clean_df = clean_df[(clean_df['glucose_level'] >= 20) & (clean_df['glucose_level'] <= 600)]
    filtered_count = initial_count - len(clean_df)
    if filtered_count > 0:
        print(f"Filtered out {filtered_count} readings with unrealistic glucose values")
    
    print(f"Final dataset: {len(clean_df)} readings")
    print(f"Date range: {clean_df['time'].min()} to {clean_df['time'].max()}")
    print(f"Glucose range: {clean_df['glucose_level'].min():.1f} to {clean_df['glucose_level'].max():.1f} mg/dL")
    
    # Save standardized data
    clean_df.to_csv(output_filename, sep='|', index=False)
    print(f"Standardized data saved to: {output_filename}")
    
    # Display sample
    print("\nSample of standardized data:")
    print(clean_df.head())
    
    return clean_df


In [ ]:
# OLGA: process cgm.txt to create patient-specific data files

def process_patient_data(cgm_df, patient_id):
    print(f"Processing data for patient: {patient_id}")
    patient_data = cgm_df[cgm_df['PtID'] == patient_id]
    
    # Save patient-specific data
    patient_filename = f"data/cgm_{patient_id}.csv"
    patient_data.to_csv(patient_filename, index=False)
    print(f"Patient data saved to: {patient_filename}")

cgmdata = "data/cgm.txt"

# Load raw file
cgm_df = pd.read_csv(
    cgmdata,
    sep="|",
)


if patient_id:
    process_patient_data(cgm_df, patient_id)
else:    
    for patient_id in cgm_df['PtID'].unique():
        process_patient_data(cgm_df, patient_id)

In [ ]:
cgm_df.columns

In [ ]:
os.makedirs("data", exist_ok=True)

# Update this path to your CGM data file
cgm_file_path = "data/cgm_10.csv"

# Standardize the CGM data
df_patient = standardize_cgm_data(cgm_file_path, "data/standardized_cgm_data.csv")

if df_patient is not None:
    print("\n" + "="*50)
    print("STANDARDIZATION COMPLETE")
    print("="*50)
    print(f"Output file: data/standardized_cgm_data.csv")
    print(f"Format: time|glucose_level")
    print(f"Records: {len(df_patient)}")
else:
    print("Standardization failed. Please check the file format and try again.")




# Clean and Pre-Process Data #

In [ ]:
# OLGA: you already cleaned the dataset
# # Clean and Pre-Process Data
# df = df.drop_duplicates()
# df = df.dropna()

# Rename columns to match existing analysis code
df_patient = df_patient.rename(columns={'time': 'DataDtTm', 'glucose_level': 'CGM'})

# OLGA this columnt is not needed, as we assume single patient data
# # Add a patient ID since standardized data doesn't have one
# df['iPtID'] = 1  # Single patient assumption

print("Standardized data loaded and cleaned:")
print(f"Date range: {df_patient['DataDtTm'].min()} to {df_patient['DataDtTm'].max()}")
print(f"Total readings: {len(df_patient)}")
print(df_patient.head())

# Seperating by Day #
Testing with one Patient

In [ ]:
# patient_id = 1
# df_patient = df[df['iPtID'] == patient_id].copy()

latest_date = df_patient['DataDtTm'].max()
start_date = latest_date - pd.Timedelta(days=30)

df_patient = df_patient[df_patient["DataDtTm"] >= start_date]
df_patient = df_patient.sort_values('DataDtTm')

print(f"Patient {patient_id} data:")
print(f"Date range: {df_patient['DataDtTm'].min()} to {df_patient['DataDtTm'].max()}")
print(f"Total readings: {len(df_patient)}")
print(f"Days of data: {(df_patient['DataDtTm'].max() - df_patient['DataDtTm'].min()).days + 1}")

df_patient.head()

In [ ]:
def create_daily_summaries_csv(df_patient, output_filename="daily_cgm_summaries.csv"):
    """Create daily summaries of CGM data and save directly as CSV"""
    
    # Add date column for grouping
    df_patient['Date'] = df_patient['DataDtTm'].dt.date
    
    # Initialize lists to store summary data
    dates = []
    # patient_ids = []
    mean_glucose = []
    std_dev = []
    glucose_range = []
    pct_high = []
    pct_low = []
    pct_in_range = []
    time_high_hours = []
    time_low_hours = []
    time_in_range_hours = []
    max_glucose = []
    min_glucose = []
    time_of_peak = []
    time_of_lowest = []
    readings_count = []
    day_of_week = []
    
    # Group by date and calculate metrics
    grouped = df_patient.groupby('Date')
    
    for date, group in grouped:
        glucose = group["CGM"]
        
        # Basic statistics
        dates.append(str(date))
        # patient_ids.append(int(group['iPtID'].iloc[0]))
        mean_glucose.append(round(glucose.mean(), 2))
        std_dev.append(round(glucose.std(), 2))
        
        # Range calculations
        max_val = glucose.max()
        min_val = glucose.min()
        max_glucose.append(max_val)
        min_glucose.append(min_val)
        glucose_range.append(max_val - min_val)
        
        # Percentage calculations
        pct_high.append(round((glucose > 180).mean() * 100, 2))
        pct_low.append(round((glucose < 70).mean() * 100, 2))
        pct_in_range.append(round(((glucose >= 70) & (glucose <= 180)).mean() * 100, 2))
        
        # Time-based analysis (assuming 5-minute intervals)
        time_high_hours.append(round((glucose > 180).sum() * 5 / 60, 1))
        time_low_hours.append(round((glucose < 70).sum() * 5 / 60, 1))
        time_in_range_hours.append(round(((glucose >= 70) & (glucose <= 180)).sum() * 5 / 60, 1))
        
        # Peak and lowest times
        time_of_peak.append(group.loc[glucose.idxmax(), "DataDtTm"].strftime("%H:%M"))
        time_of_lowest.append(group.loc[glucose.idxmin(), "DataDtTm"].strftime("%H:%M"))
        
        # Additional metrics
        readings_count.append(len(glucose))
        day_of_week.append(pd.to_datetime(date).strftime("%A"))
    
    # Create DataFrame directly
    daily_summaries_df = pd.DataFrame({
        'Date': dates,
        #'PatientID': patient_id,
        'DayOfWeek': day_of_week,
        'MeanGlucose': mean_glucose,
        'StdDev': std_dev,
        'GlucoseRange': glucose_range,
        'PercentHigh': pct_high,
        'PercentLow': pct_low,
        'PercentInRange': pct_in_range,
        'TimeHighHours': time_high_hours,
        'TimeLowHours': time_low_hours,
        'TimeInRangeHours': time_in_range_hours,
        'MaxGlucose': max_glucose,
        'MinGlucose': min_glucose,
        'TimeOfPeak': time_of_peak,
        'TimeOfLowest': time_of_lowest,
        'ReadingsCount': readings_count
    })
    
    # Save to CSV
    daily_summaries_df.to_csv(output_filename, index=False)
    print(f"Daily summaries saved to {output_filename}")
    
    return daily_summaries_df

# Generate the CSV file
daily_df = create_daily_summaries_csv(df_patient, "data/daily_cgm_summaries.csv")

# Display first few rows
print(f"Generated {len(daily_df)} daily summaries:")
print("\nFirst 3 days summary:")
for i in range(min(3, len(daily_df))):
    row = daily_df.iloc[i]
    print(f"\nDate: {row['Date']} ({row['DayOfWeek']})")
    print(f"Mean Glucose: {row['MeanGlucose']} mg/dL")
    print(f"Time in Range (70-180): {row['PercentInRange']}% ({row['TimeInRangeHours']} hours)")
    print(f"Time High (>180): {row['PercentHigh']}% ({row['TimeHighHours']} hours)")
    print(f"Time Low (<70): {row['PercentLow']}% ({row['TimeLowHours']} hours)")
    print(f"Peak: {row['MaxGlucose']} at {row['TimeOfPeak']}")

# Display the DataFrame
daily_df.head()

In [ ]:
def create_period_summary_csv(daily_df, days, output_filename=None):
    """Create summary for last N days and save as CSV"""
    

    # Adjust days if we don't have enough data
    if len(daily_df) < days:
        days = len(daily_df)
    
    # Get the most recent N days
    # OLGA: sort by date to ensure we get the last N days correctly
    daily_df.sort_values(by='Date', ascending=True, inplace=True) 
    recent_days = daily_df.tail(days)
    
    # Calculate averages using pandas operations
    mean_glucose = round(recent_days['MeanGlucose'].mean(), 2)
    mean_std = round(recent_days['StdDev'].mean(), 2)
    mean_pct_high = round(recent_days['PercentHigh'].mean(), 2)
    mean_pct_low = round(recent_days['PercentLow'].mean(), 2)
    mean_pct_in_range = round(recent_days['PercentInRange'].mean(), 2)
    
    # Calculate glucose variability (std of daily means)
    cgm_variability = round(recent_days['MeanGlucose'].std(), 2)
    
    # Calculate average time metrics
    avg_time_high = round(recent_days['TimeHighHours'].mean(), 2)
    avg_time_low = round(recent_days['TimeLowHours'].mean(), 2)
    avg_time_in_range = round(recent_days['TimeInRangeHours'].mean(), 2)
    
    # Get max and min values
    max_daily_glucose = recent_days['MaxGlucose'].max()
    min_daily_glucose = recent_days['MinGlucose'].min()
    
    # Create summary DataFrame
    summary_df = pd.DataFrame({
        'Period': [f"Last {days} days"],
        'StartDate': [recent_days['Date'].iloc[0]],
        'EndDate': [recent_days['Date'].iloc[-1]],
        'MeanGlucose': [mean_glucose],
        'GlucoseVariability': [cgm_variability],
        'MeanStdDev': [mean_std],
        'AvgPercentHigh': [mean_pct_high],
        'AvgPercentLow': [mean_pct_low],
        'AvgPercentInRange': [mean_pct_in_range],
        'AvgTimeHighHours': [avg_time_high],
        'AvgTimeLowHours': [avg_time_low],
        'AvgTimeInRangeHours': [avg_time_in_range],
        'MaxGlucoseObserved': [max_daily_glucose],
        'MinGlucoseObserved': [min_daily_glucose],
        'TotalDays': [days]
    })
    
    # Save to CSV if filename provided
    if output_filename:
        summary_df.to_csv(output_filename, index=False)
        print(f"Period summary saved to {output_filename}")
    
    return summary_df

# Generate summaries
summary_7_days_df = create_period_summary_csv(daily_df, 7, "data/7_day_summary.csv")
summary_30_days_df = create_period_summary_csv(daily_df, 30, "data/30_day_summary.csv")

# Display results
print("7-Day Summary:")
for col in summary_7_days_df.columns:
    print(f"{col}: {summary_7_days_df[col].iloc[0]}")

print("\n" + "="*50)
print("30-Day Summary:")
for col in summary_30_days_df.columns:
    print(f"{col}: {summary_30_days_df[col].iloc[0]}")

# Display the DataFrames
print("\n7-Day Summary DataFrame:")
print(summary_7_days_df)

print("\n30-Day Summary DataFrame:")
print(summary_30_days_df)

In [ ]:
def format_for_llm_analysis_csv(daily_df, patient_id, summary_7_days_df, summary_30_days_df, output_filename="cgm_analysis_report.txt"):
    """Format CGM data in a readable format for LLM analysis"""
    
    # Extract values from DataFrames
    # patient_id = daily_df['PatientID'].iloc[0]
    start_date = summary_30_days_df['StartDate'].iloc[0]
    end_date = summary_30_days_df['EndDate'].iloc[0]
    
    # Create the LLM report text
    llm_report = f"""
CGM DATA ANALYSIS REPORT
Patient ID: {patient_id}
Analysis Period: {start_date} to {end_date}

RECENT PERIOD SUMMARIES:

Last 7 Days:
- Average Glucose: {summary_7_days_df['MeanGlucose'].iloc[0]} mg/dL
- Time in Range (70-180 mg/dL): {summary_7_days_df['AvgPercentInRange'].iloc[0]}%
- Time Above Range (>180 mg/dL): {summary_7_days_df['AvgPercentHigh'].iloc[0]}%
- Time Below Range (<70 mg/dL): {summary_7_days_df['AvgPercentLow'].iloc[0]}%
- Glucose Variability: {summary_7_days_df['GlucoseVariability'].iloc[0]} mg/dL
- Average Daily High Time: {summary_7_days_df['AvgTimeHighHours'].iloc[0]} hours
- Average Daily Low Time: {summary_7_days_df['AvgTimeLowHours'].iloc[0]} hours

Last 30 Days:
- Average Glucose: {summary_30_days_df['MeanGlucose'].iloc[0]} mg/dL
- Time in Range (70-180 mg/dL): {summary_30_days_df['AvgPercentInRange'].iloc[0]}%
- Time Above Range (>180 mg/dL): {summary_30_days_df['AvgPercentHigh'].iloc[0]}%
- Time Below Range (<70 mg/dL): {summary_30_days_df['AvgPercentLow'].iloc[0]}%
- Glucose Variability: {summary_30_days_df['GlucoseVariability'].iloc[0]} mg/dL
- Highest Glucose Observed: {summary_30_days_df['MaxGlucoseObserved'].iloc[0]} mg/dL
- Lowest Glucose Observed: {summary_30_days_df['MinGlucoseObserved'].iloc[0]} mg/dL

DAILY BREAKDOWN (Last 14 Days):
"""

    # Get last 14 days for detailed breakdown
    recent_14_days = daily_df.tail(14)
    
    # Add daily details for pattern recognition
    for idx, row in recent_14_days.iterrows():
        llm_report += f"""
{row['Date']} ({row['DayOfWeek']}):
  Mean: {row['MeanGlucose']} mg/dL | Range: {row['MinGlucose']}-{row['MaxGlucose']} mg/dL
  Time in Range: {row['PercentInRange']}% | High: {row['PercentHigh']}% | Low: {row['PercentLow']}%
  Peak at {row['TimeOfPeak']} ({row['MaxGlucose']} mg/dL) | Lowest at {row['TimeOfLowest']} ({row['MinGlucose']} mg/dL)
  Standard Deviation: {row['StdDev']} mg/dL
"""
    
    # Save the text report
    with open(output_filename, "w") as f:
        f.write(llm_report)
    print(f"Report saved to '{output_filename}'")
    
    return llm_report

# Generate the analysis
# OLGA: moved output file to data subfolder for consistency
llm_analysis_text = format_for_llm_analysis_csv(daily_df, patient_id, summary_7_days_df, summary_30_days_df, "data/cgm_analysis_report.txt")

print("LLM Analysis Report Generated:")
print("="*60)
print(llm_analysis_text[:1000] + "..." if len(llm_analysis_text) > 1000 else llm_analysis_text)